In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored
import asyncio

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://requestor-agent:3021 and an api key of adminApiKey


In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


In [4]:
listeners = []

# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")
    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))



connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

agent_controller.register_listeners(listeners,defaults=True)

## Create Invite Before Requesting Mediation



In [5]:
unmediated_invitation = await agent_controller.connections.create_invitation()

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  ef96fb3b-c499-477c-b040-16f3ffc4cafb
State :  invitation
----------------------------------------------------------
invitation


## Make Connection with Mediator

In [6]:
mediator_invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '0ea5e233-7d65-41b6-bf39-d22c3b6bc6f8', 'label': 'MediatorAgent', 'serviceEndpoint': 'https://8dee682d2e61.ngrok.io', 'recipientKeys': ['3cVWmAw4Y6gLt2e2B3qEHuXHKx3rQh761F4XZ5tFgXsZ']}



In [7]:
response = await agent_controller.connections.receive_invitation(mediator_invitation)

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  633dc49f-f502-45f8-88c3-b7e044adb465
State :  invitation
----------------------------------------------------------
invitation
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  633dc49f-f502-45f8-88c3-b7e044adb465
State :  request
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  633dc49f-f502-45f8-88c3-b7e044adb465
State :  response
----------------------------------------------------------
response
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  633dc49f-f502-45f8-88c3-b7e044adb465
State :  active
----------------------------------------------------------
Connection ID: 633dc49f-f502-45f8-88c3-b7e044adb465 is now active.


In [8]:
connection_id = response["connection_id"]
print(connection_id)

633dc49f-f502-45f8-88c3-b7e044adb465


### Request mediation

In [9]:
### check state of connection
connection = await agent_controller.connections.get_connection(connection_id)

# Let's check for the state
if connection['state'] != 'active':
    print("No active connection. \n Please go back and ensure you have established an active connection between the mediator agent and Alice's subwallet agent")    
else:
    ## request mediation
    mediation_req = await agent_controller.mediation.request_mediation(connection_id)
    print(mediation_req)
    


{'updated_at': '2021-06-04 15:58:23.162699Z', 'role': 'client', 'created_at': '2021-06-04 15:58:23.162699Z', 'connection_id': '633dc49f-f502-45f8-88c3-b7e044adb465', 'mediator_terms': [], 'recipient_terms': [], 'state': 'request', 'mediation_id': '457f2eea-bd2a-458f-8773-05ec180adf3b', 'routing_keys': []}


## Check Mediation Records

State should be "granted" if mediator agent has accepted the mediation request

In [10]:
### To have a look at the mediation records and we should see our mediation in there
response = await agent_controller.mediation.get_mediation_records()
print(response)
print(f"State : {response[0]['state']}")

[{'updated_at': '2021-06-04 15:58:23.511646Z', 'role': 'client', 'created_at': '2021-06-04 15:58:23.162699Z', 'connection_id': '633dc49f-f502-45f8-88c3-b7e044adb465', 'mediator_terms': [], 'recipient_terms': [], 'state': 'granted', 'mediation_id': '457f2eea-bd2a-458f-8773-05ec180adf3b', 'routing_keys': ['4HAkhmf5cJ5V9LHLs3jE8VDrufNaJQEiegE7nz3E8S9h'], 'endpoint': 'https://8dee682d2e61.ngrok.io'}]
State : granted


## Set Default Mediator

Your agent will automatically use this mediator when creating connections

In [ ]:
default_mediation_res = await agent_controller.mediation.set_default_mediator(response[0]['mediation_id'])
print(default_mediation_res)

## Create Mediated Connection Invitation

In [ ]:
mediated_conn_invitation = await agent_controller.connections.create_invitation()


## See the Difference Between Invitations

The mediated connection invitation uses the routing keys and service endpoint provided by the mediator.

In [ ]:
mediated_routing_keys = mediated_conn_invitation["invitation"]["routingKeys"]
mediated_endpoint = mediated_conn_invitation["invitation"]["serviceEndpoint"]
unmediated_routing_key = unmediated_invitation["invitation"]["routingKeys"]
unmediated_endpoint = unmediated_invitation["invitation"]["serviceEndpoint"]

print("Before Mediation")
print(f"Endpoint : {unmediated_endpoint}, Routing Keys : {unmediated_routing_key}")
print("After Mediation")
print(f"Endpoint : {mediated_endpoint}, Routing Keys : {mediated_routing_keys}")

## Terminate Controller

In [12]:
await agent_controller.terminate()